In [ ]:
%cd ../../

# Train sig generator

In [ ]:
from ml_hep_sim.pipeline.prebuilt.flow_pipeline import FlowPipeline

In [ ]:
sig_override = {
    "datasets": {
        "data_name": "higgs_sig",
        "data_params": {
            "subset_n": [10 ** 6, 10 ** 5, 10 ** 5],
            "rescale": "logit_normal",
            "to_gpu": True,
        },
    },
    "logger_config": {"run_name": "Higgs_glow", "experiment_name": "analysis"},
    "trainer_config": {"gpus": 1, "max_epochs": 51},
    "model_config": {"num_flows": 10},
}

FP_sig = FlowPipeline(
    run_name="Higgs_Glow_sig",
    model_name="Glow",
    override=sig_override,
    pipeline_path="ml_pipeline/analysis/Higgs_glow/",
)

FP_sig.build_train_pipeline()
FP_sig.fit()

# Train bkg generator

In [ ]:
import copy

In [ ]:
bkg_override = copy.deepcopy(sig_override)
bkg_override["datasets"]["data_name"] = "higgs_bkg"

FP_bkg = FlowPipeline(
    run_name="Higgs_Glow_bkg",
    model_name="Glow",
    override=bkg_override,
    pipeline_path="ml_pipeline/analysis/Higgs_glow/",
)

FP_bkg.build_train_pipeline()
FP_bkg.fit()

# Build sig and bkg generators using inference pipelines

In [ ]:
N_gen = 10 ** 6

FP_sig.build_inference_pipeline(N_gen, rescale_data=False, device="cuda")
FP_bkg.build_inference_pipeline(N_gen, rescale_data=False, device="cuda")

sig_infer_pipeline = FP_sig.pipeline["inference_pipeline"]
bkg_infer_pipeline = FP_bkg.pipeline["inference_pipeline"]

# Get MC sig data block

In [ ]:
from ml_hep_sim.pipeline.blocks import DatasetBuilderBlock, ReferenceDataLoaderBlock

In [ ]:
mc_sig_config = copy.deepcopy(FP_sig.pipeline["train_pipeline"].pipes[0]) # same config as before
mc_sig_config.config["datasets"]["data_params"]["subset_n"] = [0, 0, N_gen]
mc_sig_config.config["datasets"]["data_params"]["rescale"] = "none"

b_mc_sig_dataset = DatasetBuilderBlock()(mc_sig_config)
b_mc_sig_data = ReferenceDataLoaderBlock(rescale_reference="logit_normal", device="cpu")(b_mc_sig_dataset)

# Get MC bkg data block

In [ ]:
mc_bkg_config = copy.deepcopy(FP_bkg.pipeline["train_pipeline"].pipes[0])
mc_bkg_config.config["datasets"]["data_params"]["subset_n"] = [0, 0, N_gen]
mc_bkg_config.config["datasets"]["data_params"]["rescale"] = "none"

b_mc_bkg_dataset = DatasetBuilderBlock()(mc_bkg_config)
b_mc_bkg_data = ReferenceDataLoaderBlock(rescale_reference="logit_normal", device="cpu")(b_mc_bkg_dataset)

# Train binary classifier

In [ ]:
from ml_hep_sim.pipeline.prebuilt.classifier_pipeline import ClassifierPipeline

In [ ]:
override = {
    "datasets": {
        "data_name": "higgs",
        "data_params": {
            "subset_n": [10 ** 6, 10 ** 5, 10 ** 5],
            "rescale": "logit_normal",
            "to_gpu": True,
        },
    },
    "logger_config": {"run_name": "Higgs_classifier", "experiment_name": "analysis"},
    "trainer_config": {"gpus": 1, "max_epochs": 101},
    "model_config": {
        "resnet": False,
        "hidden_layers": [256, 128, 64, 1],
    },
}

CP = ClassifierPipeline(
    "Higgs_classifier", override=override, pipeline_path="ml_pipeline/analysis/classifiers/"
)

CP.build_train_pipeline()
CP.fit(force=False)

# Load trained classifier

In [ ]:
from ml_hep_sim.pipeline.blocks import ModelLoaderBlock

In [ ]:
class_train_pipeline = CP.pipeline["train_pipeline"]

config = class_train_pipeline.pipes[0] # classifier config block
model = class_train_pipeline.pipes[3] # classifier model trainer block

b_classifier_model = ModelLoaderBlock(device="cuda")(config, model)

# Use classifier

In [ ]:
from ml_hep_sim.pipeline.blocks import ClassifierRunnerBlock

In [ ]:
b_flow_sig_generated = sig_infer_pipeline.pipes[-1]
b_flow_bkg_generated = bkg_infer_pipeline.pipes[-1]

b_sig_gen_class = ClassifierRunnerBlock(save_data=False, device="cuda")(b_flow_sig_generated, b_classifier_model)  # sig gen
b_bkg_gen_class = ClassifierRunnerBlock(save_data=False, device="cuda")(b_flow_bkg_generated, b_classifier_model)  # bkg gen

b_sig_mc_class = ClassifierRunnerBlock(save_data=False, device="cuda")(b_mc_sig_data, b_classifier_model)  # MC sig
b_bkg_mc_class = ClassifierRunnerBlock(save_data=False, device="cuda")(b_mc_bkg_data, b_classifier_model)  # MC bkg

# Use variable

In [ ]:
from ml_hep_sim.pipeline.blocks import VariableExtractBlock
from ml_hep_sim.analysis.utils import get_colnames_dict

In [ ]:
var = "m bb"

dct = get_colnames_dict()
idx = dct[var]

In [ ]:
b_sig_gen_var = VariableExtractBlock(idx, save_data=False, device="cuda")(b_flow_sig_generated) # sig gen var
b_bkg_gen_var = VariableExtractBlock(idx, save_data=False, device="cuda")(b_flow_bkg_generated) # bkg gen var

b_sig_mc_var = VariableExtractBlock(idx, save_data=False, device="cuda")(b_mc_sig_data) # MC sig var
b_bkg_mc_var = VariableExtractBlock(idx, save_data=False, device="cuda")(b_mc_bkg_data) # MC bkg var

# Build and fit 

In [ ]:
from ml_hep_sim.pipeline.pipes import Pipeline

In [ ]:
pipe = Pipeline()
pipe.compose(
    b_mc_sig_dataset,
    b_mc_sig_data,
    b_mc_bkg_dataset,
    b_mc_bkg_data,
    b_classifier_model,
    sig_infer_pipeline,
    bkg_infer_pipeline,
    b_sig_gen_var,
    b_bkg_gen_var,
    b_sig_mc_var,
    b_bkg_mc_var,
    b_sig_gen_class,
    b_bkg_gen_class,
    b_sig_mc_class,
    b_bkg_mc_class,
)
pipe.fit()

In [ ]:
pipe.pipes[6]#.trained_model.__class__.__name__

In [ ]:
pipe.draw_pipeline_tree(to_graphviz_file="pipeline_mc", block_idx=-1)
pipe.draw_pipeline_tree(to_graphviz_file="pipeline_gen", block_idx=-3)

pipe.draw_pipeline_tree(to_graphviz_file="pipeline_gen_cut", block_idx=-7)

# Plot histograms - classifier

In [ ]:
import matplotlib.pyplot as plt
from ml_hep_sim.plotting.style import style_setup, set_size

set_size()
style_setup(seaborn_pallete=True)

In [ ]:
sig_gen = pipe.pipes[-4].results
bkg_gen = pipe.pipes[-3].results
sig_mc = pipe.pipes[-2].results[: len(sig_gen)]
bkg_mc = pipe.pipes[-1].results[: len(sig_gen)]

In [ ]:
plt.hist(sig_gen, histtype="step", range=(-0.5, 1.25), bins=40, lw=2)
plt.hist(bkg_gen, histtype="step", range=(-0.5, 1.25), bins=40, lw=2)
plt.hist(sig_mc, histtype="step", range=(-0.5, 1.25), bins=40, lw=2)
plt.hist(bkg_mc, histtype="step", range=(-0.5, 1.25), bins=40, lw=2)
plt.legend(["sig gen", "bkg gen", "sig mc", "bkg mc"], loc="upper left")
plt.show()

# Plot histograms - variable

In [ ]:
sig_gen = pipe.pipes[-1-4].results
bkg_gen = pipe.pipes[-2-4].results
sig_mc = pipe.pipes[-3-4].results[: len(sig_gen)]
bkg_mc = pipe.pipes[-4-4].results[: len(sig_gen)]

In [ ]:
plt.hist(sig_gen, histtype="step", range=(-5, 5), bins=40, lw=2)
plt.hist(bkg_gen, histtype="step", range=(-5, 5), bins=40, lw=2)
plt.hist(sig_mc, histtype="step", range=(-5, 5), bins=40, lw=2)
plt.hist(bkg_mc, histtype="step", range=(-5, 5), bins=40, lw=2)
plt.legend(["sig gen", "bkg gen", "sig mc", "bkg mc"], loc="upper left")
plt.xlabel("$m_{bb}$ logit normal space")
plt.show()

In [ ]:
break

# Plot all distributions
change rescale_data to False and use None rescaling instead of logit_normal for this

In [ ]:
from ml_hep_sim.stats.stat_plots import N_sample_plot
from ml_hep_sim.data_utils.higgs.process_higgs_dataset import LATEX_COLNAMES

In [ ]:
sig_gen = pipe.pipes[1].reference_data.cpu().numpy()
bkg_gen = pipe.pipes[3].reference_data.cpu().numpy()
sig_mc = pipe.pipes[7].generated_data[: len(sig_gen)].cpu().numpy()
bkg_mc = pipe.pipes[10].generated_data[: len(sig_gen)].cpu().numpy()

In [ ]:
BIN_RANGES = [
    [0, 4],
    [-3, 3],
    [-0.1, 4],
    [0, 5],
    [-4, 4],
    [0, 4],
    [-5, 5],
    [0, 5],
    [-4, 4],
    [0, 5],
    [-3, 3],
    [0, 3],
    [0, 3],
    [0.75, 1.5],
    [0, 3],
    [0, 3],
    [0, 3],
    [0, 3],
]

In [ ]:
fig, axs = plt.subplots(6, 3, figsize=(13, 19))
axs = axs.flatten()

res = [sig_gen, bkg_gen, sig_mc, bkg_mc]

N_sample_plot(res, axs, n_bins=40, log_scale=False, 
              labels=LATEX_COLNAMES, lw=2, alpha=1, 
              label=["sig gen", "bkg gen", "sig mc", "bkg mc"],
              xlim=BIN_RANGES, bin_range=BIN_RANGES)
plt.tight_layout()